Descobrindo a senha com DEAP
============================



## Importações



In [2]:
import random
import numpy as np

from deap import base
from deap import creator
from deap import tools
from deap.algorithms import eaSimple

from funcoes import gene_letra

## Problema da senha usando `DEAP`



Primeiro definimos as constantes.



In [3]:
# relacionadas ao problema a ser resolvido
SENHA = "correcthorsebatterystaple"
LETRAS_POSSIVEIS = "abcdefghijklmnopqrstuvwxyz"
NUM_GENES = len(SENHA)

# relacionadas à busca
TAMANHO_POP = 50
NUM_GERACOES = 100
CHANCE_CRUZAMENTO = 0.5
CHANCE_MUTACAO = 0.15
NUM_COMBATENTES_NO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

Precisamos definir a função objetivo também.



In [4]:
def funcao_objetivo_senha(individuo, senha_verdadeira):
    """Computa a funcao objetivo de um individuo no problema da senha

    Args:
      individiuo: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    Returns:
      Uma tupla com a  "distância" entre a senha proposta e a senha verdadeira.
      Essa distância é medida letra por letra. Quanto mais distante uma letra
      for da que deveria ser, maior é essa distância.
    """
    diferenca = 0

    for letra_candidato, letra_oficial in zip(individuo, senha_verdadeira):
        diferenca = diferenca + abs(ord(letra_candidato) - ord(letra_oficial))

    return (diferenca, )

# definindo a nossa querida função_objetivo
# igual a que já conhecemos, mas com a definição da tupla

O `DEAP` não tem uma função de mutação de letras, precisamos usar a que nós criamos na disciplina. Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Não sei dizer o motivo dessa escolha, temos que aceitar e seguir.



In [5]:
def mutacao_senha(individuo, letras):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      letras: letras possíveis de serem sorteadas.

    Return:
      Uma tupla de um individuo (senha) com um gene mutado.
    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_letra(letras)
    return (individuo, )

O restante não tem segredo. Único detalhe é que agora o problema é de minimização, então precisamos alterar isso.



In [6]:
# Devemos alterar aqui por ser um problema de minimização
# por isso, precisamos reescrever ele
creator.create("Fitness_min", base.Fitness, weights=(-1.0,)) # -1 para minimização
creator.create("Individual", list, fitness=creator.Fitness_min) # só p/ lembrar que é minimização mesmo

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_letra, LETRAS_POSSIVEIS) # registrando a criação das letras
toolbox.register(
    "individual",
    tools.initRepeat,
    creator.Individual,
    toolbox.cria_gene,
    NUM_GENES,
)
toolbox.register(
    "population", tools.initRepeat, list, toolbox.individual, TAMANHO_POP
)
toolbox.register("evaluate", funcao_objetivo_senha, SENHA)
toolbox.register(
    "select", tools.selTournament, tournsize=NUM_COMBATENTES_NO_TORNEIO
)
toolbox.register("mate", tools.cxOnePoint)
toolbox.register(
    "mutate",
    mutacao_senha,
    letras=LETRAS_POSSIVEIS
)

hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("avg", np.mean)
estatisticas.register("std", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.population()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_CRUZAMENTO,
    mutpb=CHANCE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

print()
print("Hall da fama:")
print("".join(hall_da_fama.items[0]))

# podemos usar criando uma maneira de parar em 100 (gerações)

gen	nevals	avg   	std    	min	max
0  	50    	224.18	26.8631	149	279
1  	22    	198.5 	24.1812	146	237
2  	35    	180.46	21.284 	142	218
3  	38    	161.74	13.4266	128	191
4  	26    	153.36	10.0574	132	171
5  	17    	143.6 	7.52596	123	162
6  	30    	137.82	7.21579	122	157
7  	25    	133.12	3.675  	125	142
8  	26    	130.5 	6.07042	114	155
9  	26    	126.56	6.11281	114	140
10 	27    	121.32	7.04398	101	135
11 	30    	116.22	7.17855	102	136
12 	28    	110.66	6.21807	96 	129
13 	23    	107.42	6.08963	96 	123
14 	25    	101.94	5.11629	92 	120
15 	32    	97.98 	4.87233	88 	117
16 	32    	95.02 	3.72822	86 	108
17 	23    	92.5  	4.47772	80 	105
18 	33    	90.02 	5.37211	77 	108
19 	27    	86.56 	5.08787	77 	99 
20 	28    	83.2  	5.51362	74 	100
21 	26    	79.52 	4.63569	65 	97 
22 	33    	77.2  	3.45832	65 	90 
23 	27    	74.36 	4.26502	64 	84 
24 	31    	71.44 	4.34585	63 	79 
25 	32    	69.2  	5.93296	63 	91 
26 	34    	65.72 	4.39108	62 	84 
27 	27    	64.48 	3.90507	62 	82 
28 	33    	64.